## Homework

In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the 
[previous homework](../08-deep-learning/homework.md).

Download the model from here: 

https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras

In [1]:
import tensorflow as tf
from tensorflow import keras

2024-12-11 15:22:06.414666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 15:22:06.433041: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 15:22:06.438284: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 15:22:06.453307: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-11 15:22:07.895679: W tensorflow/compiler/tf2

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 27 Mb
* 43 Mb
* 77 Mb
* 127 Mb

In [2]:
model = keras.models.load_model('./model_2024_hairstyle.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('hair-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

I0000 00:00:1733905416.630753    3940 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733905416.744642    3940 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733905416.744708    3940 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733905416.748702    3940 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733905416.748770    3940 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

INFO:tensorflow:Assets written to: /tmp/tmpjrp05gfa/assets


INFO:tensorflow:Assets written to: /tmp/tmpjrp05gfa/assets


Saved artifact at '/tmp/tmpjrp05gfa'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140059527808272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140059527825888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140059524058976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140059524101040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140059524124160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140059524123104: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1733905424.284576    3940 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1733905424.285381    3940 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-12-11 15:23:44.289166: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpjrp05gfa
2024-12-11 15:23:44.289746: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-11 15:23:44.289758: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpjrp05gfa
2024-12-11 15:23:44.296603: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-12-11 15:23:44.297704: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-11 15:23:44.428810: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpjrp05gfa
2024-12-11 15:23:44.435785: I tensorflow/cc/saved_model/loader.cc

In [4]:
!ls -lh ./hair-model.tflite | awk '{print $5,$9}'

77M ./hair-model.tflite


## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* 13
* 24

In [5]:
interpreter = tf.lite.Interpreter(model_path="hair-model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_index = input_details[0]['index']
output_index = output_details[0]['index']

print("Input index:", input_index)
print("Output index:", output_index)


Input index: 0
Output index: 13


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

Let's download and resize this image: 

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

In [6]:
from io import BytesIO
from urllib import request
from PIL import Image

In [7]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [8]:
img = download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg')

In [10]:
prep_img = prepare_image(img, (200,200))

## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.24
* 0.44
* 0.64
* 0.84

In [16]:
import numpy as np

def rescale_img(img):
    img_arr = np.array(img)
    return img_arr / 255.0
    

In [18]:
rescaled_img_arr = rescale_img(prep_img)

In [19]:
rescaled_img_arr[0, 0, 0]

0.23921568627450981

## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.293
* 0.493
* 0.693
* 0.893

In [29]:
X = np.expand_dims(rescaled_img_arr, axis=0).astype('float32')
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [34]:
preds

array([[0.8934675]], dtype=float32)

In [32]:
preds[0].tolist()

[0.8934674859046936]

## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10

COPY model_2024_hairstyle_v2.tflite .

RUN pip install numpy==1.23.1
```

Note that it uses Python 3.10. The latest models of TF Lite
do not support Python 3.12 yet, so we need to use 3.10. Also,
for this part, we will use TensorFlow 2.14.0. We have tested
it, and the models created with 2.17 could be served with 2.14.0.

For that image, we also needed to use an older version of numpy
(1.23.1)

The docker image is published to [`agrigorev/model-2024-hairstyle:v3`](https://hub.docker.com/r/agrigorev/model-2024-hairstyle/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The wheel for this combination that you'll need to use in your Docker image is https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl


## Question 5

Download the base image `agrigorev/model-2024-hairstyle:v3`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

So what's the size of this base image?

* 182 Mb
* 382 Mb
* 582 Mb
* 782 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.


In [33]:
!docker images | grep hairstyle

agrigorev/model-2024-hairstyle   v3        607850a2138e   8 days ago     782MB


## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `model_2024_hairstyle_v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

* 0.229
* 0.429
* 0.629
* 0.829


In [35]:
!curl http://localhost:8080/2015-03-31/functions/function/invocations -d '{"url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}'

[0.42985352873802185]

## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.


## Publishing to Docker hub

Just for the reference, this is how we published our image to Docker hub:

```bash
docker build -t model-2024-hairstyle -f homework.dockerfile .
docker tag model-2024-hairstyle:latest agrigorev/model-2024-hairstyle:v3
docker push agrigorev/model-2024-hairstyle:v3
```
